In [1]:
!pip install altair


In [2]:
# Imports
import pandas as pd
import json
import csv
import altair as alt

In [3]:
# Path to Reviews Dataset, change to local path
reviews_path = "D:\\Yelp-JSON\\Yelp JSON\\yelp_dataset\\yelp_academic_dataset_review.json"

reviews = []

# Loop over each line in json file, avoid memory errors
with open(reviews_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        obj = json.loads(line)

        # Keep only business_id and text
        review = {
            "business_id": obj["business_id"],
            "text": obj["text"]
        }

        reviews.append(review)

    
        if (i + 1) % 500000 == 0:
            print(f" Loaded {i+1} reviews...")

            
# Convert List to DF
reviewsDF = pd.DataFrame(reviews)
print(f"\n Final dataset shape: {reviewsDF.shape}")


 Loaded 500000 reviews...
 Loaded 1000000 reviews...
 Loaded 1500000 reviews...
 Loaded 2000000 reviews...
 Loaded 2500000 reviews...
 Loaded 3000000 reviews...
 Loaded 3500000 reviews...
 Loaded 4000000 reviews...
 Loaded 4500000 reviews...
 Loaded 5000000 reviews...
 Loaded 5500000 reviews...
 Loaded 6000000 reviews...
 Loaded 6500000 reviews...

 Final dataset shape: (6990280, 2)


In [17]:
# Check first 5 results
print(reviewsDF.head())

              business_id                                               text
0  XQfwVwDr-v0ZS3_CbbE5Xw  If you decide to eat here, just be aware it is...
1  7ATYjTIgM3jUlt4UM3IypQ  I've taken a lot of spin classes over the year...
2  YjUWPpI6HXG530lwP-fb2A  Family diner. Had the buffet. Eclectic assortm...
3  kxX2SOes4o-D3ZQBkiMRfA  Wow!  Yummy, different,  delicious.   Our favo...
4  e4Vwtrqf-wpJfwesgvdgxQ  Cute interior and owner (?) gave us tour of up...


In [5]:
# Path to Businesses Dataset, change to local path
businesses_path = "D:\\Yelp-JSON\\Yelp JSON\\yelp_dataset\\yelp_academic_dataset_business.json"

# Load in data
businessesDF = pd.read_json(businesses_path, lines=True)

# Filter to only needed columns
businessesDF = businessesDF[["business_id", "city", "state"]]

# Check DF shape
print(f"Final shape: {businessesDF.shape}")


Final shape: (150346, 3)


In [6]:
# Check first 5 results
print(businessesDF.head())

              business_id           city state
0  Pns2l4eNsfO8kk83dixA6A  Santa Barbara    CA
1  mpf3x-BjTdTEA3yCZrAYPw         Affton    MO
2  tUFrWirKiKi_TAnsVWINQQ         Tucson    AZ
3  MTSW4McQd7CbVtyjqoe9mw   Philadelphia    PA
4  mWMc6_wTdE0EUBKIGXDVfA     Green Lane    PA


In [7]:
# Merge reviews with city and state information using business_id
reviewLocationDF = pd.merge(reviewsDF, businessesDF, on="business_id", how="inner")

# Print first 5 results
print(reviewLocationDF.head())

              business_id                                               text  \
0  XQfwVwDr-v0ZS3_CbbE5Xw  If you decide to eat here, just be aware it is...   
1  XQfwVwDr-v0ZS3_CbbE5Xw  This is the second time we tried turning point...   
2  XQfwVwDr-v0ZS3_CbbE5Xw  The place is cute and the staff was very frien...   
3  XQfwVwDr-v0ZS3_CbbE5Xw  We came on a Saturday morning after waiting a ...   
4  XQfwVwDr-v0ZS3_CbbE5Xw  Mediocre at best. The decor is very nice, and ...   

          city state  
0  North Wales    PA  
1  North Wales    PA  
2  North Wales    PA  
3  North Wales    PA  
4  North Wales    PA  


In [8]:
# Count by states and set columns
state_counts = reviewLocationDF["state"].value_counts().reset_index()
state_counts.columns = ["state", "review_count"]

# Filter by > 50,000 reviews and only U.S. states
valid_states = state_counts[(state_counts["review_count"] >= 50000) & (state_counts["state"] != "AB")]["state"]

# Filter the original DF
reviewLocationDF = reviewLocationDF[reviewLocationDF["state"].isin(valid_states)]

# Recount reviews by state
state_counts = reviewLocationDF["state"].value_counts().reset_index()
state_counts.columns = ["state", "review_count"]
state_counts = state_counts.sort_values(by="review_count", ascending=False)

# Print Results
print(f"Filtered shape: {reviewLocationDF.shape}")
print("States included:", reviewLocationDF['state'].unique())
print("\nFinal review counts by state:")
print(state_counts)


Filtered shape: (6880548, 4)
States included: ['PA' 'AZ' 'LA' 'CA' 'FL' 'IN' 'MO' 'TN' 'NV' 'NJ' 'IL' 'ID' 'DE']

Final review counts by state:
   state  review_count
0     PA       1598960
1     FL       1161545
2     LA        761673
3     TN        614388
4     MO        502385
5     IN        489752
6     AZ        431708
7     NV        430678
8     CA        348856
9     NJ        260897
10    ID        157572
11    DE         70302
12    IL         51832


In [9]:
city_counts = reviewLocationDF.groupby(["city"]).size().reset_index(name="review_count")

# Sort by review count descending
city_counts = city_counts.sort_values(by="review_count", ascending=False)

# Print top results
print(city_counts.head(20))

                  city  review_count
843       Philadelphia        967552
736        New Orleans        635364
1122             Tampa        454847
723          Nashville        451566
1174            Tucson        404865
494       Indianapolis        361489
902               Reno        351573
981      Santa Barbara        269630
963        Saint Louis        253437
86               Boise        105366
196         Clearwater         87442
968   Saint Petersburg         78907
1042            Sparks         73033
658           Metairie         64361
1074         St. Louis         63451
368           Franklin         56783
1080    St. Petersburg         54480
405             Goleta         45643
1293        Wilmington         44814
100            Brandon         43464


In [10]:
# Define state to region mapping
# https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf

state_to_region = {
    
    # Northeast
    "PA": "Northeast",
    "NJ": "Northeast",
    "DE": "Northeast",

    # Midwest
    "IL": "Midwest",
    "IN": "Midwest",
    "MO": "Midwest",

    # South
    "FL": "South",
    "LA": "South",
    "TN": "South",

    # West
    "AZ": "West",
    "NV": "West",
    "CA": "West",
    "ID": "West"
}

# Use map to create region column
reviewLocationDF["region"] = reviewLocationDF["state"].map(state_to_region)

# Count reviews by region
region_counts = reviewLocationDF["region"].value_counts().reset_index()
region_counts.columns = ["region", "review_count"]

# Display results
print(region_counts)




      region  review_count
0      South       2537606
1  Northeast       1930159
2       West       1368814
3    Midwest       1043969


In [18]:
# Check first 5 results
print(reviewLocationDF.head())

              business_id                                               text  \
0  XQfwVwDr-v0ZS3_CbbE5Xw  If you decide to eat here, just be aware it is...   
1  XQfwVwDr-v0ZS3_CbbE5Xw  This is the second time we tried turning point...   
2  XQfwVwDr-v0ZS3_CbbE5Xw  The place is cute and the staff was very frien...   
3  XQfwVwDr-v0ZS3_CbbE5Xw  We came on a Saturday morning after waiting a ...   
4  XQfwVwDr-v0ZS3_CbbE5Xw  Mediocre at best. The decor is very nice, and ...   

          city state     region  review_length  
0  North Wales    PA  Northeast            513  
1  North Wales    PA  Northeast            477  
2  North Wales    PA  Northeast            216  
3  North Wales    PA  Northeast            736  
4  North Wales    PA  Northeast            953  


In [12]:
# Remove the city column, only looking at state and region
exportDF = reviewLocationDF.drop(columns=["city"])

# Export to CSV and compress
# Uncomment to download, takes a while...

# exportDF.to_csv(
#     "reviews_with_locations.csv.gz",
#     index=False,
#     encoding="utf-8",
#     quoting=csv.QUOTE_ALL,
#     compression="gzip"
# )

In [13]:
# State Bar Chart
stateBars = alt.Chart(state_counts).mark_bar().encode(
    y=alt.Y("state:N", title="State", sort="-x"),
    x=alt.X("review_count:Q", axis=None), 
)

# Labels for Review Counts
stateLabels = alt.Chart(state_counts).mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    y=alt.Y("state:N", sort="-x"),
    x=alt.X("review_count:Q"),
    text=alt.Text("review_count:Q")
)

# Combine State Bar Chart & Labels
stateReviewBar = (stateBars + stateLabels).configure_view(
    stroke=None
).properties(
    title="Number of Yelp Reviews by State",
    width=600,
    height=400
)

stateReviewBar


alt.LayerChart(...)

In [14]:
# Group by city and get top 20 with most reviews
city_counts = reviewLocationDF.groupby("city").size().reset_index(name="review_count")
city_counts = city_counts.sort_values(by="review_count", ascending=False).head(20)

# Bar Chart for Cities
cityBars = alt.Chart(city_counts).mark_bar().encode(
    y=alt.Y("city:N", title="City", sort="-x"),
    x=alt.X("review_count:Q", axis=None)
)

# Labels for City Review Counts
cityLabels = alt.Chart(city_counts).mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    y=alt.Y("city:N", sort="-x"),
    x=alt.X("review_count:Q"),
    text=alt.Text("review_count:Q")
)

# Combine City Bar Chart & Labels
cityReviewBar = (cityBars + cityLabels).configure_view(
    stroke=None
).properties(
    title="Top 20 Cities by Number of Yelp Reviews",
    width=600,
    height=500
)

cityReviewBar


alt.LayerChart(...)

In [15]:
# Region Bars
regionBars = alt.Chart(region_counts).mark_bar().encode(
    y=alt.Y("region:N", sort="-x", title="Region"),
    x=alt.X("review_count:Q", axis=None),
    tooltip=["region", "review_count"]
)

# Region Labels for Review Count
regionLabels = alt.Chart(region_counts).mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    y=alt.Y("region:N", sort="-x"),
    x=alt.X("review_count:Q"),
    text=alt.Text("review_count:Q")
)

# Combine Region Bar Chart and Labels
region_chart = (regionBars + regionLabels).configure_view(
    stroke=None
).properties(
    title="Yelp Review Counts by U.S. Region",
    width=500,
    height=300
)

region_chart

alt.LayerChart(...)

In [16]:
# Compute review length and add column
reviewLocationDF["review_length"] = reviewLocationDF["text"].str.len()

# Total average
avg_total_length = reviewLocationDF["review_length"].mean()
print(f"Average review length overall: {avg_total_length:.2f} characters")

# Average by state
avg_by_state = reviewLocationDF.groupby("state")["review_length"].mean().reset_index()
avg_by_state.columns = ["state", "avg_review_length"]
print("\nAverage review length by state:")
print(avg_by_state.sort_values(by="avg_review_length", ascending=False))

# Average by region
avg_by_region = reviewLocationDF.groupby("region")["review_length"].mean().reset_index()
avg_by_region.columns = ["region", "avg_review_length"]
print("\nAverage review length by region:")
print(avg_by_region.sort_values(by="avg_review_length", ascending=False))


Average review length overall: 565.31 characters

Average review length by state:
   state  avg_review_length
10    NV         595.608698
11    PA         595.496253
8     MO         587.354543
6     IN         582.976976
9     NJ         570.436479
0     AZ         560.591430
12    TN         547.309487
7     LA         546.985718
2     DE         545.767119
4     ID         537.979292
1     CA         535.217735
3     FL         533.711379
5     IL         518.229858

Average review length by region:
      region  avg_review_length
1  Northeast         590.297677
0    Midwest         581.868946
3       West         562.539365
2      South         540.988009
